## Image encoder

In [44]:
from tools.image_process import getLayerNames, checkLayerNames, image2feature
import glob

img_embed = []
for fileName in sorted(glob.glob('images/*.svg')):
    # base, ext = os.path.splitext(fileName)
    # if ext == '.svg' and base != 'test' and base != 'stack':
    if 'test' in fileName or 'stack' in fileName:
        continue
    print(fileName)
    # print(get_id_list(fileName))
    names = getLayerNames(fileName)
    checkLayerNames(names)
    print(names)
    # print(image2feature(names))
    img_embed.append(image2feature(names))
    
img_embed = np.array(img_embed)
print(img_embed.shape)

images/1.svg
['A2122', 'A3212']
images/10.svg
['A1', 'A2121_1_', 'A323']
images/11.svg
['A1', 'A2121_1_', 'A313']
images/2.svg
['A4', 'A2121', 'A324']
images/3.svg
['A4', 'A2122', 'A312']
images/4.svg
['A2211', 'A311']
images/5.svg
['A2212', 'A3211']
images/6.svg
['A2112', 'A311']
images/7.svg
['A2121', 'A324']
images/8.svg
['A311', 'A2121']
images/9.svg
['A2121']
(11, 37)


In [3]:
# Caveats
# The first <image> element in a <g> is the shadow, namely the extra .png file in the dir

In [4]:
## test corpus
test_text = ['A man is lying on the sofa.',
             'A man is sitting next to a computer.',
             'A man is presenting a chart. The light is on. we',
             'A woman is standing next to a bucket.',
             "It's a deer in a sock",
             'A christmas tree with presents']

## text encoder

In [3]:
import glob
from tools.text_process import LemmaTokenizer
tokenizer = LemmaTokenizer()
sentences = []
for fileName in sorted(glob.glob('text/*.txt')):
    print(fileName)
    with open(fileName, 'r') as f:
        sent = f.read()
        print(sent)
        tokens = tokenizer(sent)
        print(tokens)
        print('----')
        sentences.append(tokens)

text/1.txt
A man looks at his chart with his back.

['man.n.01', 'look.v.01', 'chart.n.01', 'back.n.01']
----
text/10.txt
A man walks past a web Icon
['man.n.01', 'walk.v.01', 'past.n.01', 'web.n.01', 'icon.n.01']
----
text/11.txt
A man and a woman are interacting
['man.n.01', 'woman.n.01', 'be.v.01', 'interact.v.01']
----
text/2.txt
A man demonstrates the chart.
['man.n.01', 'show.v.01', 'chart.n.01']
----
text/3.txt
A man and a woman play circles with compasses.
['man.n.01', 'woman.n.01', 'play.n.01', 'circle.n.01', 'compass.n.01']
----
text/4.txt
A girl was leaning against a car.
['girl.n.01', 'be.v.01', 'lean.v.01', 'car.n.01']
----
text/5.txt
A girl was standing on the edge of the city.
['girl.n.01', 'be.v.01', 'stand.v.01', 'edge.n.01', 'city.n.01']
----
text/6.txt
A man is cleaning his computer.
['man.n.01', 'be.v.01', 'clean.v.01', 'computer.n.01']
----
text/7.txt
A boy is demonstrating a form.
['male_child.n.01', 'be.v.01', 'show.v.01', 'form.n.01']
----
text/8.txt
There are t

#### vectorizer

In [4]:
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer

vectorizer = TfidfVectorizer(ngram_range=(1,2),
                             norm=None,
                             sublinear_tf=True,
                             stop_words=[],
                             lowercase=False,
                             tokenizer=lambda l: l)
text_embed = vectorizer.fit_transform(sentences)
print(type(text_embed))
print(text_embed.shape)
# print(vectorizer.fit_transform(sentences).toarray())
print(len(vectorizer.vocabulary_))

<class 'scipy.sparse.csr.csr_matrix'>
(11, 64)
64


#### Similarity between category keywords and the description

In [5]:
from tools.text_process import keywordSimi
import numpy as np

In [9]:
kw_simis = []
for sentence in sentences:
    print(sentence)
    kw_simis.append(keywordSimi(sentence))

kw_simis = np.array(kw_simis)

['man.n.01', 'look.v.01', 'chart.n.01', 'back.n.01']
['man.n.01', 'walk.v.01', 'past.n.01', 'web.n.01', 'icon.n.01']
['man.n.01', 'woman.n.01', 'be.v.01', 'interact.v.01']
['man.n.01', 'show.v.01', 'chart.n.01']
['man.n.01', 'woman.n.01', 'play.n.01', 'circle.n.01', 'compass.n.01']
['girl.n.01', 'be.v.01', 'lean.v.01', 'car.n.01']
['girl.n.01', 'be.v.01', 'stand.v.01', 'edge.n.01', 'city.n.01']
['man.n.01', 'be.v.01', 'clean.v.01', 'computer.n.01']
['male_child.n.01', 'be.v.01', 'show.v.01', 'form.n.01']
['be.v.01', 'three.n.01', 'hallmark.n.01', 'about.r.03', 'person.n.01']
['mobile.n.01', 'telephone.n.01', 'chart.n.01']


#### all

In [7]:
import scipy

In [10]:
print(text_embed.shape)
print(kw_simis.shape)
text_embed = scipy.sparse.hstack([text_embed, kw_simis])
print(text_embed.shape)

(11, 97)
(11, 33)
(11, 130)


### random image generator

## Training

In [47]:
scipy.sparse.hstack([text_embed, img_embed]).shape

(11, 134)

## Renderer

In [42]:
def str2num_size(size):
    assert type(size) is str
    return int(float(size.strip('pt')))
    
def get_size(file):
    """
    Get the width and height of an image
    
    .svg:
        use xml's parse
        By check the 'viewBox' attribute in the 'svg' tag
    
    .png:
        use PIL
    """
    
    basename, ext = os.path.splitext(file)
    if ext == '.svg':   
        from xml.dom.minidom import parse
        doc = parse(file)
        # search svg element
        image_list = doc.getElementsByTagName('svg')
        assert(image_list), 'no svg element found!'
        assert(len(image_list) == 1), file
        img = image_list[0]
        assert(img.hasAttribute('viewBox'))
        width, height = img.getAttribute('viewBox').split()[2:]
        width, height = (str2num_size(width), str2num_size(height))
        # try search image element, then g element
#     try:
#         assert image_list
#     except AssertionError:
#         image_list = doc.getElementsByTagName('svg')
#         assert image_list
#         assert image_list[0].hasAttribute('width')
#     assert(len(image_list) == 1), svg_file
#     ## if multiple images, size is the maximum size in either direction        
    elif ext == '.png':
        from PIL import Image
        width, height = Image.open(file).size

    return (width, height)

In [43]:
def stack_svgs(file_list, opt_file=None, canvas_size=None):
 
    import cairosvg
    from svgutils.compose import Figure, Image #,SVG
    
    if not opt_file:
        opt_file = 'stack.svg'
        
    if canvas_size:
        canvas_w, canvas_h = canvas_size
    else:
        canvas_w, canvas_h = 0, 0
        for file in file_list:
            width, height = get_size(file)
            if width > canvas_w: canvas_w = width
            if height > canvas_h: canvas_h = height
    print('Canvas size:', (canvas_w, canvas_h))
    
    # if svg, convert to png first
    file_list_png = []
    for file in file_list:
        basename, ext = os.path.splitext(file)
        if ext == '.svg':
            png_file = basename + '.png'
            cairosvg.svg2png(url=file,
                             write_to=png_file)
            file_list_png.append(png_file)
        elif ext == '.png':
            file_list_png.append(file)
        else:
            raise ValueError('File type not availale!')
    
    image_list = []
    # 
    for file in file_list_png:
        print('File:', file)
        width, height = get_size(file)
        img = Image(width, height, file)
        img.move(int((canvas_w-width)/2),
                 int((canvas_h-height)/2))
        image_list.append(img)
    
        
    Figure(canvas_w, canvas_h, *image_list).save(opt_file)

In [46]:
stack_svgs(['svg-objects-export/A-1.png', 
            'svg-objects-export/A-2-1-2-1.png', 
            'svg-objects-export/A-3-1.png'])

Canvas size: (1114, 786)
File: svg-objects-export/A-1.png
File: svg-objects-export/A-2-1-2-1.png
File: svg-objects-export/A-3-1.png


In [45]:
stack_svgs(['material/背景4.svg', 
            'material/123.svg', 
            'material/生活方式3.svg'])

Canvas size: (1145, 816)
File: material/背景4.png
File: material/123.png
File: material/生活方式3.png


/Users/dongjustin/miniconda2/envs/text2scene/lib/python3.7/site-packages/cairosvg/parser.py:416: UserWarning: No tag with id=未命名的渐变_12-2 found.
  warnings.warn('No tag with id={} found.'.format(element_id))
/Users/dongjustin/miniconda2/envs/text2scene/lib/python3.7/site-packages/cairosvg/parser.py:416: UserWarning: No tag with id=未命名的渐变_12-3 found.
  warnings.warn('No tag with id={} found.'.format(element_id))
/Users/dongjustin/miniconda2/envs/text2scene/lib/python3.7/site-packages/cairosvg/parser.py:416: UserWarning: No tag with id=未命名的渐变_12-4 found.
  warnings.warn('No tag with id={} found.'.format(element_id))


In [13]:
## write a tool to stack two svgs:
### put <g> altogether, then for each elememt, add attribute recursively

In [354]:
## ValueError: material/124.svg
## issue of cairosvg converter

## Test

In [132]:
doc = parse('SB.svg')
# ele = [c for c in doc.childNodes if c.nodeType == 1]
# assert(len(ele) == 1)
# [n.getAttribute('id') for n in ele[0].childNodes if n.nodeType==1 and n.tagName=='g']
# [g.getAttribute('id') for g in doc.getElementsByTagName('g') if g.hasAttribute('id')]
# id_list = [g.getAttribute('id') for g in doc.getElementsByTagName('g') if g.hasAttribute('id')]
# assert(id_list)

# get the tagname: element.tagName

In [76]:
# the order may be reversed, that's weird
# at least. background must be at the bottom
# but what if there are only person and surronding

# it's reversed
# because the first object is at the bottom

In [74]:
doc.getElementsByTagName('image')

[<DOM Element: image at 0x112681cc0>]

In [56]:
[g.getAttribute('id') for g in doc.getElementsByTagName('*') if g.hasAttribute('id')]

['_x31__x5F_1_x5F_1']